# Extract and display a connectivity matrix from a SONATA circuit

Copyright (c) 2025 Open Brain Institute

Authors: Christoph Pokorny

Last modified: 01.2025

## Summary
This analysis extracts and visualizes a matrix of connection probabilities or #synapses per connection (mean/SD/...), grouped by a selected neuron property (layer, m-type, ...).
For details, see the [README](README.md).

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

from bluepysnap import Circuit
from connectome_manipulator.connectome_comparison import connectivity
from ipywidgets import widgets, interact

## Circuit selection

A SONATA circuit is assumed to be located under `./analysis_circuit/circuit_config.json`. For using an existing circuit from some other location, modify the `circuit_config = ...` path below accordingly.

In [4]:
# Path to existing circuit config
circuit_config = "analysis_circuit/circuit_config.json"

assert os.path.exists(circuit_config), f"ERROR: Circuit config '{os.path.split(circuit_config)[1]}' not found!"

Loading SONATA circuit. Selections of the edge population containing the synapses, as well as pre-/post-synaptic node sets defining groups of neurons are possible.

In [5]:
c = Circuit(circuit_config)
e_populations = c.edges.population_names
assert len(e_populations) > 0, "ERROR: No edge population found!"
node_sets = list(c.node_sets.content.keys())
e_popul_wdgt = widgets.Dropdown(options=e_populations, description="Edge population:", style={"description_width": "auto"}, layout=widgets.Layout(width="max-content"))
pre_nset_wdgt = widgets.Dropdown(options=[None] + node_sets, description="Pre-synaptic node set:", style={"description_width": "auto"}, layout=widgets.Layout(width="max-content"))
post_nset_wdgt = widgets.Dropdown(options=[None] + node_sets, description="Post-synaptic node set:", style={"description_width": "auto"}, layout=widgets.Layout(width="max-content"))
display(e_popul_wdgt)
display(pre_nset_wdgt)
display(post_nset_wdgt)

Dropdown(description='Edge population:', layout=Layout(width='max-content'), options=('nodeA__nodeA__chemical'…

Dropdown(description='Pre-synaptic node set:', layout=Layout(width='max-content'), options=(None, 'LayerA', 'L…

Dropdown(description='Post-synaptic node set:', layout=Layout(width='max-content'), options=(None, 'LayerA', '…

The connectivity is grouped by one selected (categorical) neuron properties, like layer, m-type, etc.

In [6]:
def get_props(nodes):
    is_cat = nodes.get([]).dtypes.apply(lambda _x: isinstance(_x, pd.CategoricalDtype))
    props = is_cat[is_cat].index.to_list()  # Select categorical properties
    for _p in ["layer", "mtype", "etype", "synapse_class"]:
        if _p in nodes.property_names:
            props = props + [_p]  # Add certain properties, even if not categorical
    return np.unique(props)
node_props = np.intersect1d(get_props(c.edges[e_popul_wdgt.value].source), get_props(c.edges[e_popul_wdgt.value].target))
groupby_wdgt = widgets.Dropdown(options=node_props, value="layer" if "layer" in node_props else node_props[0], description="Group-by:", style={"description_width": "auto"}, layout=widgets.Layout(width="max-content"))
display(groupby_wdgt)

Dropdown(description='Group-by:', index=1, layout=Layout(width='max-content'), options=(np.str_('etype'), np.s…

Extract the adjacency/synaptome matrix using functionality from [connectome-manipulator](https://github.com/openbraininstitute/connectome-manipulator).

In [7]:
conn_dict = connectivity.compute(c, sel_src=pre_nset_wdgt.value, sel_dest=post_nset_wdgt.value, edges_popul_name=e_popul_wdgt.value, group_by=groupby_wdgt.value)

INFO: Computing connectivity (group_by=layer, sel_src={}, sel_dest={}, N=3x3 groups)


100% |#########################################################################|


Interactive visualization of adjacency or synaptome matrix. Marker scaling and alpha (transparency) can be adjusted for better display of small/large matrices.

In [8]:
# Interactive plot function
def plot_fct(res_sel):
    connectivity.plot(conn_dict[res_sel], conn_dict["common"], vmin=0.0, group_by=groupby_wdgt.value)

In [9]:
res_sel_opt = [(_v["unit"], _k) for _k, _v in sorted(conn_dict.items()) if "unit" in _v]
res_sel_wdgt = widgets.Dropdown(options=res_sel_opt, description="Display:", style={"description_width": "auto"}, layout=widgets.Layout(width="max-content"))
iplot = interact(plot_fct, res_sel=res_sel_wdgt)

interactive(children=(Dropdown(description='Display:', layout=Layout(width='max-content'), options=(('Conn. pr…